# Import

In [1]:
# Basic imports
import pandas as pd
import numpy as np

# Warnings
import warnings 
warnings.simplefilter("ignore")

In [2]:
# Plot
from IPython import display
import seaborn as sns

import matplotlib
import matplotlib.pylab as plt
from jupyterthemes import jtplot

jtplot.style('gruvboxd')
matplotlib.use('nbagg')

# Data Reading

In [3]:
# Data Reading
df_train = pd.read_csv('./data/house/train.csv', index_col='Id')
df_test = pd.read_csv('./data/house/test.csv', index_col='Id') 

# Split X_train, y_train
target = 'SalePrice'
features = df_test.columns

y_train = df_train[target]
df_train = df_train[features]

df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


# Preprocessing

In [2]:
from robusta.preprocessing.category import *
from robusta.preprocessing.numeric import *
from robusta.preprocessing import *
from robusta.pipeline import *

prep_pipe = make_pipeline(
    ColumnSelector(columns=features),
    FeatureUnion([
        ("numeric", make_pipeline(
            TypeSelector(np.number),
            Imputer(strategy="median"),
            GaussRank(),
        )),
        ("category", make_pipeline(
            TypeSelector("object"),
            LabelEncoder(dtype='category'),
        )),
    ])
)

X_train = prep_pipe.fit_transform(df_train)
X_test = prep_pipe.transform(df_test)

X_train.head()

NameError: name 'features' is not defined

# Fold Preparation

In [5]:
%%time
from lightgbm import LGBMRegressor
from sklearn.compose import TransformedTargetRegressor

lgbm = LGBMRegressor()
lgbm = TransformedTargetRegressor(lgbm, func=np.log1p, inverse_func=np.expm1)
lgbm.fit(X_train, y_train)

/Users/pechyonkinb/anaconda/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


CPU times: user 1.97 s, sys: 178 ms, total: 2.14 s
Wall time: 380 ms


In [1]:
from sklearn.model_selection import RepeatedKFold

cats = TypeSelector(['category', 'object'])

te = FastEncoderCV(cv=4, smooth=1)

fold_pipe = make_pipeline(cats, te)

NameError: name 'TypeSelector' is not defined

In [49]:
fold_pipe = FeatureUnion([
    ('category', make_pipeline(
        TypeSelector(['category', 'object']), 
        FastEncoderCV(cv=4, smoothing=1e3))),
    ('numeric', TypeSelector(np.number))
])
    
fold_pipe.fit_transform(X_train, y_train).sample(5, random_state=555)

,Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,CentralAir,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1349,5.518611e+06,5.600278e+06,5.533287e+06,7.621656e+06,7.034820e+06,5.562248e+06,6.102728e+06,5.604412e+06,5.559152e+06,5.449626e+06,...,0.172882,1.365757,0.003642,-0.126319,-0.013962,-0.069919,-0.003642,-0.030967,0.524039,-0.361037
1110,5.518611e+06,5.600278e+06,5.533287e+06,5.887471e+06,7.034820e+06,5.562248e+06,9.753106e+06,5.604412e+06,5.559152e+06,5.449626e+06,...,1.202419,0.270502,1.197297,-0.126319,-0.013962,-0.069919,-0.003642,-0.030967,-0.922824,0.596492
1354,5.518611e+06,5.600278e+06,5.533287e+06,6.164613e+06,7.034820e+06,5.562248e+06,6.102728e+06,5.604412e+06,5.559152e+06,5.449626e+06,...,0.710390,0.980035,0.612598,-0.126319,-0.013962,-0.069919,-0.003642,-0.030967,-0.922824,4.320005
224,5.465960e+06,5.536015e+06,5.466977e+06,4.918970e+06,4.838807e+06,4.899945e+06,4.197684e+06,5.542792e+06,5.480907e+06,5.390031e+06,...,0.461804,0.726666,-0.534159,-0.126319,-0.013962,-0.069919,-0.003642,-0.030967,-0.922824,0.596492
907,5.544071e+06,5.593891e+06,5.541222e+06,4.964971e+06,7.215715e+06,5.568473e+06,6.090048e+06,5.614530e+06,5.577089e+06,5.472129e+06,...,1.220863,-0.452825,0.396122,-0.126319,-0.013962,-0.069919,-0.003642,-0.030967,-0.053148,-0.361037


In [52]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import *

def rmsle_score(y_true, y_pred): 
    return -np.sqrt(mean_squared_log_error(y_true, y_pred))

def rmsle_scorer(estimator, X_test, y_test): 
    return rmsle_score(y_test, estimator.predict(X_test))

cv = 5
scoring = rmsle_scorer

#param = 'encodercv__encoder__smoothing'
#a, b = 0, 10
#param_space = np.logspace(a, b, int(b-a+1))

#gs = GridSearchCV(model_pipe_te, {param: param_space}, cv=cv, scoring=scoring)
#gs.fit(X_train, y_train)


#model_pipe_te = make_pipeline(cats, te, lgbm)
#model_pipe_te_opt = make_pipeline(cats, te, lgbm)

estimator = LGBMRegressor()

model = make_pipeline(fold_pipe, estimator)
model_tt = TransformedTargetRegressor(model)

scores = cross_val_score(model_tt, X_train, y_train, cv=cv, scoring=scoring)

print('{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'